# Кластеризация текстов

## Типы данных

In [1]:
#документ имя -> слово -> встречаемость слова
#документ это словарь слов, с количеством встреч в этом документе
Document = dict[str,int]
#Слово это строка
Word = str
#Слова это словарь слов с их весом
Vector = dict[str,float]
#Текст это список слов
Text = list[Word]
# матрица ассоциативности слов
Matrix = dict[Word,dict[Word,float]]


## Функции импорта докуметов из пдф файлов , субд


In [2]:
from os import listdir
from os.path import isfile, join
import PyPDF2 as ppdf

In [3]:
def text(file):
    file = open(file,'rb')
    reader = ppdf.PdfFileReader(file,strict = False)
    text =""
    for page in reader.pages:
        text += page.extract_text()
    return text

def texts(directory : str) -> dict[str,str]:
    files = [f for f in listdir(directory) if isfile(join(directory, f))] 
    res = dict[str,str].fromkeys(files,"")
    #todo распаралелить , но в питоне это как-то слишком странно представлено 
    for file in files:
        try:
            res[file] = text(join(directory, file))
        except:
            print(f'exception on {file} while attamted to read it')
    return res

## Обработка текстов

In [4]:
import nltk
from nltk.stem import LancasterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import pandas as pd
from numpy import linalg as LA
import math
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network
import itertools
from networkx.algorithms.components import connected_components

Удаление спецсимволов , Стопслов , Токенизация и Стемминг


In [5]:
def removePunctuations(text : Text) -> Text:
    return [word for word in text if word.isalnum()]

def removeStopWords(text : Text) -> Text:
    stops = stopwords.words('english')
    return [word for word in text if word not in stops]

def tokenize(text : str) -> Text : 
    return word_tokenize(text)

def stemm (text : Text) -> Text:
    stemer = LancasterStemmer()
    return list(map(lambda w : stemer.stem(w),text))

def removeSmallValueTokens(text :Text,minLenght : int) -> Text:
    return [t for t in text if len(t) > minLenght]

Частота слов в документе / документах

In [6]:
def frequency(text: Text) -> Document:
    res = dict[str, int].fromkeys(text, 0)
    for word in text:
        res[word] += 1
    return res


def normFrequency(text: Text) -> Vector:
    prev = frequency(text)
    res = Vector.fromkeys(prev.keys(),0.0)
    for k, v in prev.items():
        res[k] = prev[k] / len(text)
    return Vector(sorted(res.items(),key = lambda item : -item[1]))

# возвращает отсортированый по невозрастанию словарь список
def docFrequency(document: Document) -> Document:
    res = dict[str, int]()
    for word in document.keys():
        if (res.__contains__(word) is False):
            res[word] = 1
        else:
            res[word] += 1
    return dict[str, int](sorted(res.items(), key=lambda item: -item[1]))


Обработчик любого текста по умолчанию

In [7]:
def defaultTextPipe(inputText : str) -> Text:
    text = inputText.lower()
    text = tokenize(text)
    text = removePunctuations(text)
    text = removeStopWords(text)
    text = stemm(text)
    # text = removeSmallValueTokens(text,2)
    return text

## Обработка Документов

Образ документа 

In [8]:
def docView(
        #выбранные слова в документе -> нормальзированный вес слова
        document : Document,
        #слово -> количество документов, в котором оно встречается
        commonWords : Document,
        #количество документов
        documentCount : int
    ) -> Vector:
    
    # пока что раскроем скобки
    def a(key) :
        if(document.__contains__(key) is False): return 0.0
        return document[key]  * math.log(documentCount / commonWords[key])
    w = Vector()
    for word in commonWords: w[word] = a(word)
    norm = LA.norm(w.values(),2)
    d = Vector()
    for word in d.keys(): d[word] = w[word] / norm
    return d

Расстояние между документами

In [9]:
def dist(d1: Vector, d2: Vector) -> float:
    return max(list(map(lambda x: abs(x[0] - x[1]), zip(d1.values(), d2.values()))))

Выборка терминов из Т

In [10]:
#в данном методе не гарантируется что weightLimit не будет превышен
#ожидаемое поведение : возвращение элементов, веса которых не превышают лимит
# + элемент, который привел к привышению
def selectWords(words: Text, wightLimit: float) -> Text:
    normed = normFrequency(words)
    return selectWordsFromDictionary(normed, wightLimit)

def selectWordsFromDictionary(words: Vector, wightLimit: float) -> Text:
    total : float = 0.0
    #todo задача о рюкзаке
    #пока что супер ленивое решение
    def limitTotal(word : str):
        nonlocal total
        res : bool = total < wightLimit
        total += words[word]
        return res 
    
    return [w for w in words.keys() if limitTotal(w) is True ]

Матрица совместной встречаемости

In [11]:
def selectedDocumentsWords(ds: list[Text]) -> Text:
    res = Text()
    for t in ds:
        for w in t:
            if res.__contains__(w) is False:
                res.append(w)
    return res

# a _ij = среднее количество словарных окон заданного размера , в котором содержатся слова ij, из всех докуметов

def buildMatrix(documents: list[Text], selected: Text, distance: int) -> Matrix:
    res = Matrix.fromkeys(selected,dict[Word,float]())
    total = 0
    for w1 in res.keys():
        res[w1] = Matrix.fromkeys(selected, 0.0)
    
    for w1 in res.keys():
        for w2 in res[w1].keys():
            total = 0
            for d in documents:
                total += intercectedTimes(w1, w2, distance, d)
            res[w1][w2] = total  / len(documents)
    return res

# проверяет содержатся ли слова в заданном промежутке в документе . возвращает количество таких интервалов


def intercectedTimes(w1: Word, w2: Word, distance: int, text: Text) -> int:
    indices = [i for i, x in enumerate(text) if x == w1]
    count = 0
    for i in indices:
        r = right(i, distance, text)
        l = left(i, distance, text)
        count +=  len([w for w in r if w == w2]) + len([w for w in l if w == w2])
    return count


def right(i: int, distance: int, text: Text) -> Text:
    f_i1 = i + 1 if ((i + 1) <= len(text)) else len(text)
    f_i2 = i + distance + 1 if i + distance + 1 <= len(text) else len(text)
    return text[f_i1:f_i2: +1]


def left(i: int, distance: int, text: Text) -> Text:
    b_i1 = i - 1 if ((i - 1) > -1) else 0
    b_i2 = i - 1 - distance if i - 1 - distance > -1 else 0
    if i == 0:
        return []
    if b_i2 >= b_i1 or (b_i2 == 0 and b_i1 > 0):
        return text[:b_i1 + 1]
    return text[b_i1:b_i2: -1]

Выбор центров кластеров. Центр класса это слово, с наибольшим средним весом его ребер

In [43]:
def selectCentres(matrix : Matrix,count : int) -> Text:
    collapsed = Vector()
    for w1 in matrix.keys():
        collapsed[w1] = 0.0
        edgesCount = len([v for v in matrix[w1].values() if v > 0])
        for w2 in matrix[w1].keys():
        # if w1 != w2: 
            collapsed[w1] += matrix[w1][w2]
        collapsed[w1] = collapsed[w1] / edgesCount
    # dict[str, int](sorted(res.items(), key=lambda item: -item[1]))
    v : Vector = Vector(sorted(collapsed.items(), key = lambda item : - item[1]))
    return list(v.keys())[0 : count]       

## Построение графов

In [13]:
def toNetwork(g : nx.Graph) -> Network:
    net = Network(notebook=True)
    for w in g.nodes:
        net.add_node(w,label=w,)
    for u,v in g.edges:
        net.add_edge(u,v,label=g[u][v]['weight'],value = g[u][v]['weight'])
    return net

def toGraph(matrix : Matrix) -> nx.Graph:
    res = nx.Graph()
    res.add_nodes_from(matrix.keys())
    for w1 in matrix.keys():
        for w2 in matrix[w1].keys():
            if matrix[w1][w2] != 0.0:
                res.add_edge(w1,w2,weight=matrix[w1][w2])
    return res

def show(G : nx.Graph):
    pos = nx.spring_layout(G,seed = 7)
    # pos = nx.planar_layout(G)
    # pos = nx.random_layout(G)
    
    nx.draw_networkx_nodes(G, pos, node_size=math.ceil(10000/len(G.nodes())))
    nx.draw_networkx_edges(G, pos, edgelist=G.edges())
    nx.draw_networkx_labels(G, pos, font_size=2, font_family="sans-serif")
    
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G,pos,edge_labels,label_pos=0.5,verticalalignment="bottom",clip_on=False)
    
    ax = plt.gca()
    ax.margins(0.08)
    plt.axis("off")
    plt.tight_layout()
    plt.show()

## Работа с графами

Удаляем связи между центрами класторов АКА строим разрезы

In [14]:
def removeWeakestConnections(g : nx.Graph, centers : Text) -> nx.Graph :
    res : nx.Graph = g.copy()
    pairs = itertools.combinations(centers,2)
    for c1,c2 in pairs :
        while True:
            try:
                path = nx.dijkstra_path(res,c1,c2)
                edges=list(zip(path[0:],path[1:]))
                edgeValues = list(map(lambda x:res[x[0]][x[1]]['weight'] ,edges))
                edgesValues = list(zip(edges,edgeValues))
                minEdge = list(sorted(edgesValues))[0]
                res.remove_edge(minEdge[0][0],minEdge[0][1])
            except nx.NetworkXNoPath: 
                # алгоритм не нашел пути
                break
    return res

### Выбираем значимые кластеры

In [15]:
def sortClustersBySize(clusters : list[nx.Graph]) -> list[nx.Graph]:
    return list(sorted(clusters,key = lambda x : -len(x.nodes())))

по числу

In [16]:
def clustersByCount(clusters: list[nx.Graph], count: int) -> list[nx.Graph]:
    return sortClustersBySize(clusters)[0:count]

по минимальному размеру кластера

In [17]:
def clustersByMinimumSize(clusters: list[nx.Graph], size: int) -> list[nx.Graph]:
    return [c for c in clusters if len(c.nodes()) >= size]

### Из языка графов переходим обратно на язык слов

In [18]:
def singleToText(g : nx.Graph) -> Text:
    return list(g.nodes())


def multipleToText(gs : list[nx.Graph]) -> list[Text]:
    return [singleToText(g) for g in gs]

### Определяем принадлежность документа к класетру

классификация по частоте встречаемых слов. возвращается вероятность принадлежаности к кластеру

In [19]:
def classify(document : Document, clusters : dict[str,Text]) -> dict[str,float]:
    res = dict[Text,float]().fromkeys(clusters.keys(),0.0)
    freqs = dict[Text,int]().fromkeys(clusters.keys(),0)
    for c in clusters.keys() :
        freqs[c] = sum(map(lambda x : document[x] if document.keys().__contains__(x) else 0,clusters[c]))
    
    totalCount = sum(freqs.values())
    
    for c in clusters :
        if totalCount == 0 : res[c] = 0.0
        else : res[c] = freqs[c] / totalCount
    return res
    

## Проанализируем данные

In [20]:
rawDocuments2 = texts("data")

exception on 76.pdf while attamted to read it


In [21]:
names2 = [d for d in rawDocuments2 if rawDocuments2[d] != '']
len(rawDocuments2) - len(names2)

3

In [22]:
#я так понимаю это очень плохой селектор и текстовый процессор, потому что он не удаляет мат формулы
selectionLimiter2 = 0.015
in_texts2 = list (map (lambda x : defaultTextPipe(rawDocuments2[x]),names2))
freqs2 = list (map (lambda x : frequency(x),in_texts2))
ss2 = list(map ( lambda x : selectWords(x,selectionLimiter2) ,in_texts2))
selected2 = selectedDocumentsWords(ss2)
selected2

['interv',
 'terv',
 'msbs',
 'equ',
 'pair',
 'encrypt',
 'gëê¹',
 'pad',
 'californ',
 'graph',
 'attack',
 'precondit',
 'bit',
 'diﬀ',
 '2328',
 'meas',
 'key',
 'root']

In [23]:
m2 = buildMatrix(in_texts2,selected2,3)

In [24]:
centres = selectCentres(m2,2)
centres

['key', 'attack']

In [25]:
g2 = toGraph(m2)

In [26]:

cluster = removeWeakestConnections(g2,centres)

net = Network(notebook=True)
net.from_nx(cluster)
net.toggle_physics(False)
net.show("cluster.html")

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


Посмотрим какие кластеры получились по отдельности

In [27]:


subClusters = [cluster.subgraph(c).copy() for c in connected_components(cluster)]
Szip = list(zip(subClusters,range(0,len(subClusters))))
for g_s in Szip:
    net = Network(notebook=True)
    net.from_nx(g_s[0])
    net.toggle_physics(False)
    net.show(f"sub_cluster_{g_s[1]}.html")

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


#  Побалуемся с работой


выбираем 2 кластера

In [28]:
selected_clusters = clustersByCount(subClusters,2)
cl_texts = dict[str,Text](zip(range(0,2),multipleToText(selected_clusters)))
cl_texts

{0: ['equ', 'graph', 'interv', '2328', 'key', 'bit', 'diﬀ', 'encrypt'],
 1: ['msbs', 'pad', 'attack', 'meas', 'terv', 'precondit', 'root', 'pair']}

Посмотрим к какому кластеру относится документ 102.pdf

In [29]:
doc = frequency(defaultTextPipe(rawDocuments2["102.pdf"]))
classify(doc,cl_texts)

{0: 0.9878048780487805, 1: 0.012195121951219513}

К какому 117.pdf

In [30]:
doc = frequency(defaultTextPipe(rawDocuments2["117.pdf"]))
classify(doc,cl_texts)

{0: 0.8768939393939394, 1: 0.12310606060606061}

Посмотрим как все документы относятся к кластерам

In [31]:
realNames = [n for n in names2 if rawDocuments2[n] != '']
docs = dict[str,dict[str,float]]([(x,classify(frequency(defaultTextPipe(rawDocuments2[x])),cl_texts)) for x in realNames])
pd.DataFrame.from_dict(docs,orient='index')

,0,1
102.pdf,0.987805,0.012195
103.pdf,0.281250,0.718750
10_B08-607.pdf,0.402027,0.597973
116.pdf,0.990291,0.009709
117.pdf,0.876894,0.123106
121.pdf,0.958763,0.041237
127.pdf,0.501326,0.498674
3DES Attack Merkle Hellman.pdf,0.775510,0.224490
62.pdf,0.000000,0.000000
64.pdf,0.294118,0.705882


А если мы добавим третий кластер )

In [32]:
selected_clusters = clustersByCount(subClusters,4)
cl_texts = dict[str,Text](zip(range(4),multipleToText(selected_clusters)))
cl_texts

{0: ['equ', 'graph', 'interv', '2328', 'key', 'bit', 'diﬀ', 'encrypt'],
 1: ['msbs', 'pad', 'attack', 'meas', 'terv', 'precondit', 'root', 'pair'],
 2: ['gëê¹'],
 3: ['californ']}

In [33]:
docs = dict[str,dict[str,float]]([(x,classify(frequency(defaultTextPipe(rawDocuments2[x])),cl_texts)) for x in realNames])
pd.DataFrame.from_dict(docs,orient='index')

,0,1,2,3
102.pdf,0.987805,0.012195,0.0,0.00000
103.pdf,0.281250,0.718750,0.0,0.00000
10_B08-607.pdf,0.402027,0.597973,0.0,0.00000
116.pdf,0.990291,0.009709,0.0,0.00000
117.pdf,0.876894,0.123106,0.0,0.00000
121.pdf,0.958763,0.041237,0.0,0.00000
127.pdf,0.501326,0.498674,0.0,0.00000
3DES Attack Merkle Hellman.pdf,0.775510,0.224490,0.0,0.00000
62.pdf,0.000000,0.000000,1.0,0.00000
64.pdf,0.217391,0.521739,0.0,0.26087


# Создадим кластеры по первым двум папкам, и посмотрим куда он отнесет докумнеты из третьей

In [34]:
selectedData = texts("forClust")
testData = texts("forTests")

exception on 76.pdf while attamted to read it


In [45]:
selectionLimiter = 0.03
wordMaxLengthConnection = 5
centresCount = 4

minWordCountForCluster = 2


# уберем из списка непрочтенные файлы
names = [n for n in selectedData if selectedData[n] != '']

in_texts = list (map (lambda x : defaultTextPipe(selectedData[x]),names))
freqs = list (map (lambda x : frequency(x),in_texts))

ss2 = list(map ( lambda x : selectWords(x,selectionLimiter) ,in_texts))
selected_words = selectedDocumentsWords(ss2)


matrix = buildMatrix(in_texts,selected_words,wordMaxLengthConnection)
centres = selectCentres(m2,centresCount)

print(centres)

graph = toGraph(matrix)

g_net = toNetwork(graph)
# g_net.toggle_physics(True)
g_net.toggle_stabilization(True)
g_net.show("init_graph.html")


clustered_graph = removeWeakestConnections(graph,centres)

cg_Net = toNetwork(clustered_graph)
# cg_Net.toggle_physics(True)
cg_Net.toggle_stabilization(True)

cg_Net.show("clustered_graph.html")

clusters = [clustered_graph.subgraph(c).copy() for c in connected_components(cluster)]

selected_clusters = clustersByMinimumSize(clusters,minWordCountForCluster)
cl_texts = dict[str,Text](zip(range(0,len(selected_clusters) + 1),multipleToText(selected_clusters)))

print(cl_texts)

realNames = [n for n in testData if testData[n] != '']
docs = dict[str,dict[str,float]]([(x,classify(frequency(defaultTextPipe(testData[x])),cl_texts)) for x in realNames])
pd.DataFrame.from_dict(docs,orient='index')

['2328', 'graph', 'interv', 'key']
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
{0: ['graph', '2328', 'key', 'bit', 'diﬀ', 'interv'], 1: ['precondit', 'attack', 'root', 'meas']}


,0,1
102.pdf,1.000000,0.000000
103.pdf,0.000000,1.000000
10_B08-607.pdf,0.641176,0.358824
116.pdf,1.000000,0.000000
117.pdf,0.879157,0.120843
121.pdf,0.933333,0.066667
127.pdf,0.721992,0.278008
3DES Attack Merkle Hellman.pdf,0.680000,0.320000
62.pdf,0.000000,0.000000
64.pdf,0.333333,0.666667


62 показывает странный результат. хочу посмотреть как он классифицируется.
Выяснилось что у него странная кодировка и это выброс. это подтверждается что в предыдущем примере, где кластыеры строились по всем докуметам для него выделился отдельный класс

In [46]:
doc = frequency(defaultTextPipe(testData["62.pdf"]))
doc

{'cbed': 1,
 'fhgjijijglkm': 1,
 'lnm': 1,
 'ncdez': 1,
 'thaznv': 2,
 'hacp': 1,
 'thazcv': 2,
 'rac': 2,
 'raf': 8,
 'f5ha': 1,
 'zns': 2,
 'avyr': 1,
 'gëêl': 62,
 'z1iæ': 4,
 'ijä': 2,
 'gjá': 25,
 'äcáeäc45': 1,
 'àù6päc': 1,
 'â7ûtöaü': 1,
 'gjþ': 1,
 'gjâ54': 3,
 'ýäc2': 4,
 'èyçw2': 2,
 '471i': 1,
 '252': 4,
 'æcçnê': 1,
 'èzçnê': 1,
 'èyé': 1,
 'ä3úc': 2,
 'iván': 1,
 'z13â5': 2,
 'ä3á¹â': 2,
 'gjâ': 40,
 'aáq': 4,
 'gjäc': 61,
 'aiygs': 1,
 '³äcáe': 1,
 'â5è': 75,
 'gjî': 1,
 'äc2': 2,
 'gjän': 22,
 'ýän2': 8,
 'g713': 1,
 'úc1iij': 2,
 'q13â5': 3,
 'ýgjâ': 19,
 'ijgs': 3,
 'ûtçaü': 3,
 '8cñdâ': 1,
 'z132': 1,
 '25à': 9,
 'gj4': 18,
 'ijgjåz1iø': 1,
 '45gyän': 4,
 '25äc': 10,
 'äeß': 1,
 'ý1347åaäc': 1,
 'èe13â7â5è': 2,
 'wiygs': 1,
 'y132713átá': 1,
 'ä3î': 1,
 'wäcátâ': 1,
 'wä3á': 2,
 'gëê¹': 78,
 'â5gyåy13id83ñd': 1,
 'änâ7äcátâ': 1,
 '45áq': 5,
 'än4': 7,
 'öq2': 3,
 'wøq': 2,
 'â5gaöe': 1,
 'b3wi': 1,
 'rtr': 1,
 'fyr': 1,
 'gáe': 1,
 '2dánän2': 3,
 '25äcøq1iø': 19,
 'â